Cargar valores de entrenamiento

In [96]:
import numpy as np
class GolfValues:
    def __init__(self, ambiente, temperatura, humedad, viento, jugar):
        self.ambiente = ambiente
        self.temperatura = temperatura
        self.humedad = humedad
        self.viento = viento
        self.jugar = jugar

def leer_archivo(nombre_archivo):
    datos = []
    with open(nombre_archivo, 'r') as archivo:
        for linea in archivo:
            valores = linea.strip().split(',')
            datos_ambiente = GolfValues(valores[0], valores[1], valores[2], valores[3], valores[4])
            datos.append(datos_ambiente)
        datos.pop(0)
    return np.array(datos)

def saveClases(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        primera_linea = archivo.readline().strip().lower().split(',')
    valores_primera_linea = np.array(primera_linea[:-1])
    return valores_primera_linea

def countObjects(objetos, atributo, valorAtributo):
    count = 0
    for obj in objetos:
        if getattr(obj, atributo) == valorAtributo:
            count += 1
    return count

def countObjectsArr(objetos, atributo, valorAtributo):
    count = []
    for obj in objetos:
        if getattr(obj, atributo) == valorAtributo:
            count.append(obj)
    return count

def countUniqueValues(objetos, atributo):
    valores_unicos = set()
    for obj in objetos:
        valor = getattr(obj, atributo)
        valores_unicos.add(valor)
    valores_unicos = list(valores_unicos)
    valores_unicos.sort()
    valuesU = np.array([valores_unicos, np.zeros(len(valores_unicos))])
    for i in range(valuesU.shape[1]):
        valuesU[1][i] = countObjects(objetos, atributo, valuesU[0][i])
    return valuesU

def entropy(objetos, FinalDesc):
    total_entr = 0
    valuesU = countUniqueValues(objetos, FinalDesc)
    for i in range(valuesU.shape[1]):
        #print(int(valuesU[1][i]), "/" ,objetos.size)
        total_class_entr = - (int(valuesU[1][i])/len(objetos))*np.log2(int(valuesU[1][i])/len(objetos)) 
        total_entr += total_class_entr
    return total_entr

def entropyCategory(objetos, atributo, finalDesc='jugar'):
    valuesU = countUniqueValues(objetos, atributo)
    valuesF = countUniqueValues(objetos, finalDesc)
    add = np.zeros((1, valuesU.shape[1]))
    for i in range(valuesU.shape[1]):
        arr = countObjectsArr(objetos, atributo, valuesU[0][i])
        v = 0
        for j in range(valuesF.shape[1]):
            n = countObjects(arr, finalDesc, valuesF[0][j])
            #print(n , "/", len(arr))
            if n != 0:
                v += -(n/len(arr))*np.log2(n/len(arr))
        add[0][i] = v
    valuesU = np.concatenate((valuesU, add), axis=0)
    return valuesU

def infoGain(objetos, atributo):
    info = entropyCategory(objetos, atributo, finalDesc='jugar')
    entropiaT = entropy(objetos, 'jugar')
    gain = 0
    
    for i in range(info.shape[1]):
        gain += (int(info[1][i]) / len(objetos)) * float(info[2][i])
        
    return  entropiaT - gain

def mostInformative(objetos, archivo='golf_data.csv'):
    columns = set(saveClases(archivo))  # Convertir a conjunto para eliminar duplicados
    max_info_gain = -1
    max_info_feature = None

    for column in columns:
        info_gain = infoGain(objetos, column)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            max_info_feature = column
    return max_info_feature  



In [97]:
def subtree(objetos, atributo, finalDesc='jugar'):
    tree = {}
    valuesU = countUniqueValues(objetos, atributo)
    class_list = countUniqueValues(objetos, finalDesc)
    for i in range(valuesU.shape[1]):
        arr = countObjectsArr(objetos, atributo, valuesU[0][i])
        assigned_to_node = False
        for j in range(class_list.shape[1]):
            n = countObjects(arr, finalDesc, class_list[0][j])
            if n == len(arr):
                tree[valuesU[0][i]] = class_list[0][j]
                objetos = [obj for obj in objetos if getattr(obj, atributo) != valuesU[0][i]]
                assigned_to_node = True
        if not assigned_to_node:
            tree[valuesU[0][i]] = "?"
    return tree, objetos


def imprimir_subarbol(arbol, nivel=0):
    for clave, valor in arbol.items():
        indentacion = "  " * nivel
        print(f"{indentacion}{clave}: {valor}")
        if isinstance(valor, dict):
            imprimir_subarbol(valor, nivel + 1)

     
'''arbol_generado, objetos_restantes = subtree(objetos,'ambiente')
imprimir_subarbol(arbol_generado)'''           


"arbol_generado, objetos_restantes = subtree(objetos,'ambiente')\nimprimir_subarbol(arbol_generado)"

In [98]:
def make_tree(root, prev_feature, objetos, atributo):
    class_list = countUniqueValues(objetos, atributo)
    if len(objetos) != 0:
        max_info = mostInformative(objetos)
        tree, train_data = subtree(objetos, max_info)
        next_root = None
        if prev_feature != None:
            root[prev_feature] = dict()
            root[prev_feature][max_info] = tree
            next_root = root[prev_feature][max_info]
        else:
            root[max_info] = tree
            next_root = root[max_info]
            
        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_data = countObjectsArr(objetos, max_info, node,)
                make_tree(next_root, node, feature_data, atributo)
                
    
def id3(objetos, atributo):
    tree = {}
    make_tree(tree, None, objetos, atributo)
    return tree
    
    
    
archivo = 'golf_data.csv' 
objetos = leer_archivo(archivo)    
tree = id3(objetos, 'jugar')
tree



{'ambiente': {'Ll': {'viento': {'N': 'S', 'S': 'N'}},
  'N': 'S',
  'S': {'humedad': {'A': 'N', 'N': 'S'}}}}

In [100]:
def predict(tree, instance):
    if not isinstance(tree, dict):  # if it is a leaf node
        return tree  # return the value
    else:
        root_node = next(iter(tree))  # getting the first key/feature name of the dictionary
        feature_value = getattr(instance, root_node)  # value of the feature
        if feature_value in tree[root_node]:  # checking the feature value in the current tree node
            return predict(tree[root_node][feature_value], instance)  # go to the next feature
        else:
            return None

def evaluate(tree, test_data, atributo):
    correct_predict = 0
    wrong_predict = 0
    for instance in test_data:
        result = predict(tree, instance)
        if result == getattr(instance, atributo):
            correct_predict += 1
        else:
            wrong_predict += 1
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy


In [101]:
archivoTest = 'PlayTennisTest.csv' 
objetosTest = leer_archivo(archivo)    

accuracy = evaluate(tree, objetosTest, 'jugar') #evaluating the test dataset

print(accuracy)

1.0


Crear valores de prueba

In [102]:
import random

numP = 5
posibles_valores = {
    'columna1': ['S', 'N', 'Ll'],
    'columna2': ['A', 'M', 'B'],
    'columna3': ['A', 'N'],
    'columna4': ['N', 'S']
}

# Generar las filas con valores aleatorios
filas = []
for _ in range(numP):
    fila = []
    for columna in posibles_valores:
        valor = random.choice(posibles_valores[columna])
        fila.append(valor)
    filas.append(fila)

# Crear objetos de tipo GolfValues y almacenarlos en objetosPrueba
objetosPrueba = []
for fila in filas:
    ambiente, temperatura, humedad, viento = fila
    objeto = GolfValues(ambiente, temperatura, humedad, viento, None)
    objetosPrueba.append(objeto)

def imprimir_objetos_prueba(objetos):
    for objeto in objetos:
        atributos = [str(getattr(objeto, attr)) for attr in objeto.__dict__]
        linea = ' '.join(atributos)
        print(linea)

# Ejemplo de uso
imprimir_objetos_prueba(objetosPrueba)

Ll B N N None
Ll M A S None
S B A S None
Ll A A N None
Ll M N S None
